In [10]:
import tensorflow_hub as hub
import tensorflow as tf
elmo = hub.load("https://tfhub.dev/google/elmo/3").signatures["default"]

2023-04-15 22:27:55.454101: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'unused_control_flow_input_91' with dtype int32
	 [[{{node unused_control_flow_input_91}}]]
2023-04-15 22:27:55.972729: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'unused_control_flow_input_93' with dtype int32
	 [[{{node unused_control_flow_input_93}}]]
2023-04-15 22:27:56.478043: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'unused_control_flow_input_81'

In [13]:

# Provide input tensor and create embeddings
input_tensor = ["my birthday is the best day of the year"]
embeddings_tensor = elmo(tf.constant(input_tensor))
embeddings_tensor["elmo"]

#, signature="default", as_dict=True)

# #with tf.Session() as sess:
#    # sess.run(tf.global_variables_initializer())
#     #embeddings = sess.run(embeddings_tensor)
# print(embeddings_tensor.shape)
# print(embeddings_tensor)


<tf.Tensor: shape=(1, 9, 1024), dtype=float32, numpy=
array([[[-0.05932228, -0.1326235 , -0.12817797, ...,  0.05492053,
          0.55205786, -0.15840803],
        [ 0.06928135,  0.28926367,  0.3717891 , ..., -0.22099999,
          0.6793192 ,  1.0836272 ],
        [ 0.0262635 ,  0.169811  , -0.1404518 , ..., -0.5370391 ,
          0.96461594,  0.7538884 ],
        ...,
        [-0.09166455,  0.19069326, -0.14435157, ..., -0.8409722 ,
          0.3632321 , -0.41222334],
        [ 0.400427  , -0.02149598,  0.38504004, ...,  0.4986915 ,
          0.07260805, -0.35825977],
        [ 0.10684166,  0.23578395, -0.43084237, ...,  0.06925954,
         -0.14734796,  0.13421628]]], dtype=float32)>

In [21]:
import torch
torch.tensor(embeddings_tensor["elmo"].numpy().tolist()).shape

torch.Size([1, 9, 1024])

In [19]:
torch.tensor(embeddings_tensor["elmo"].numpy()).mean(dim=1).shape

torch.Size([1, 1024])

In [36]:
from transformers import (
    PretrainedConfig,
    PreTrainedModel,
    PreTrainedTokenizer,
)

class ElmoConfig(PretrainedConfig):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = 1024
        self.num_hidden_layers = 1
        self.is_encoder_decoder = False
        self.architectures = ["Elmo"]

class TfElmoTokenizer(PreTrainedTokenizer):

    def __call__(
        self, text, return_tensors, truncation, return_offsets_mapping, text_target
    ):
        return text

    @staticmethod
    def from_pretrained(path):
        return TfElmoTokenizer()


class TfElmoModel(PreTrainedModel):
    def __init__(self):
        super().__init__(config=ElmoConfig())
        self.elmo_model = hub.load("https://tfhub.dev/google/elmo/3").signatures["default"]

    def forward(
        self,
        input_ids,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        labels=None,
    ):
        embeddings_tensor = self.elmo_model(tf.constant(input_ids))
        elmo_embeddings = embeddings_tensor["elmo"]
        return torch.tensor(elmo_embeddings.numpy())


    @staticmethod
    def from_pretrained(path):
        return TfElmoModel()

sequences = [
    "my birthday is the best day of the year: yes",
    "my birthday is the best day of the year: no",
]

tokenizer = TfElmoTokenizer.from_pretrained("")
model = TfElmoModel.from_pretrained("")
input = tokenizer(sequences, True, None, None, None)
model(input)

2023-04-15 22:46:14.007433: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'unused_control_flow_input_104' with dtype int32
	 [[{{node unused_control_flow_input_104}}]]


tensor([[[-0.0593, -0.1326, -0.1282,  ..., -0.0418,  0.5802, -0.1217],
         [ 0.0693,  0.2893,  0.3718,  ..., -0.2552,  0.7405,  1.0352],
         [ 0.0263,  0.1698, -0.1405,  ..., -0.5142,  0.9700,  0.7551],
         ...,
         [ 0.4004, -0.0215,  0.3850,  ...,  0.4602, -0.0890, -0.2614],
         [ 0.3009,  0.3644, -0.3623,  ..., -0.0338, -0.0043,  0.4348],
         [-0.2264,  0.3154, -0.5068,  ..., -0.0725, -0.3217, -0.1728]],

        [[-0.0593, -0.1326, -0.1282,  ..., -0.0325,  0.5926, -0.1301],
         [ 0.0693,  0.2893,  0.3718,  ..., -0.2438,  0.7836,  1.0701],
         [ 0.0263,  0.1698, -0.1405,  ..., -0.5472,  0.9500,  0.7168],
         ...,
         [ 0.4004, -0.0215,  0.3850,  ...,  0.3660, -0.0391, -0.2327],
         [ 0.3009,  0.3644, -0.3623,  ...,  0.1528, -0.1047,  0.2576],
         [ 0.1797, -0.0323, -0.7010,  ..., -0.1090, -0.0245,  0.2997]]])

In [37]:
model(input).shape

torch.Size([2, 10, 1024])

In [38]:
torch.mean(model(input), dim=1).shape

torch.Size([2, 1024])